# Imports

In [22]:
import os
import jsonlines
from dotenv import load_dotenv,find_dotenv
from datasets import load_dataset
from tqdm.notebook import tqdm
load_dotenv(find_dotenv())

# os.getenv('HUGGINGFACE_TOKEN')

True

## Datasets

In [8]:
dataset=load_dataset('dsmueller/ams_data_train_Llama-2-7B-Chat-GGUF-LLM-generic_100',split='train')

In [9]:
dataset[0]['text']

"### Human: What is the title of the symposium hosted by NASA Langley Research Center and Lockheed Martin Space Systems Company, as mentioned in the context?### Assistant:NASNCP-2006-2 14290  3 sth Aerospace Mechanisms Symposium  Compiled by  Edward A. Boesiger  Lockheed Martin Space Systems Company, Sunnyvale, California  Proceedings of a symposium hosted by  the NASA Langley Research Center and  Lockheed Martin Space Systems Company and  organized by the Mechanisms Education Association  held at the Williamsburg Maniott Hotel  Williamsburg, Virginia  May 17- 19,2006  May 2006 {'source': 'AMS_2006.pdf', 'page': 1}"

I've taken this dataset and renamed into train.csv in the training folder. I can't figure out how to use autotrain with a different filename...

# Autotrain
https://github.com/huggingface/autotrain-advanced

In [34]:
import os

# TinyLlama/TinyLlama-1.1B-Chat-v0.1
# project_name='./llms/'+'ams_data_train_'+'TinyLlama_1.1B_Chat_v0.1_100-2'
# model_name='TinyLlama/TinyLlama-1.1B-Chat-v0.1'
# repo_id='ai-aerospace/ams_data_train_Llama-2-7B-Chat-GGUF-LLM-generic-100-2'

# meta-llama/Llama-2-7b-chat-hf
project_name='./llms/'+'ams_data_train_'+'Mistral-7B-v0.1'+'-100'
model_name='mistralai/Mistral-7B-v0.1'
repo_id='ai-aerospace/'+'ams_data_train_'+'Mistral-7B-v0.1'+'-100'

os.environ["project_name"] = project_name
os.environ["model_name"] = model_name
os.environ["repo_id"] = repo_id

In [35]:
!source ../.venv/bin/activate
# !autotrain llm --train --project_name my-llm --model TinyLlama/TinyLlama-1.1B-Chat-v0.1 --data_path . --use-peft --use_int4 --learning_rate 2e-4 --train_batch_size 6 --num_train_epochs 3 --trainer sft

# The training dataset to be used must be called training.csv and be located in the data_path folder.
!autotrain llm --train \
    --project_name ${project_name} \
    --model ${model_name} \
    --data_path . \
    --use-peft \
    --learning_rate 2e-4 \
    --train_batch_size 6 \
    --num_train_epochs 3 \
    --trainer sft \
    --push_to_hub \
    --repo_id ${repo_id} \
    --token $HUGGINGFACE_TOKEN

⚠️ WARNING | 2023-12-10 22:06:55 | autotrain.cli.run_dreambooth:<module>:14 - ❌ Some DreamBooth components are missing! Please run `autotrain setup` to install it. Ignore this warning if you are not using DreamBooth or running `autotrain setup` already.
/Users/danmueller/Documents/GitHub/aerospace_chatbot/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
/Users/danmueller/Documents/GitHub/aerospace_chatbot/.venv/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
> INFO    Running LLM
> INFO    Params: Namespace(version=False, train